In [1]:
import os
import time
from datetime import datetime, timedelta
#from ctypes import *
import numpy as np
import matplotlib.pyplot as plt
import serial

from scipy.optimize import curve_fit

from apscheduler.schedulers.background import BackgroundScheduler

# GPBI communication
import pyvisa

# Thorlabs Shutter
from msl.equipment import EquipmentRecord, ConnectionRecord, Backend
from msl.equipment.resources.thorlabs import MotionControl

# Thorlabs Cube
from pylablib.aux_libs.devices import Thorlabs
stage_y = Thorlabs.KDC101("27001274")

# logging
import logging
logging.basicConfig(filename='logger.log', filemode='a', format='%(asctime)s - %(message)s', level=logging.INFO)


# GPBI Init ------------------------------------------------------------------------
rm = pyvisa.ResourceManager()
instKSM = rm.open_resource('GPIB0::15::INSTR')
instKMM = rm.open_resource('GPIB0::16::INSTR')
lakeshore = rm.open_resource('GPIB0::11::INSTR')
print("SourceMeter: ", instKSM.query("*IDN?"))
print("MultiMeter: ", instKSM.query("*IDN?"))
print("LakeShore: ", lakeshore.query("*IDN?"))


# Newport Delay Line --------------------------------------------------------------
from newportxps import NewportXPS
xps = NewportXPS('192.168.254.254', username='Administrator', password='Administrator')
dline = 'Group1'
xps.initialize_group(group=dline, home=True)

# Shutter ------------------------------------------------------------------------
os.environ['PATH'] += os.pathsep + 'C:/Program Files/Thorlabs/Kinesis'

sh07 = EquipmentRecord(
    manufacturer='Thorlabs',
    model='KSC101',
    serial='68000117',  # update the serial number for your KSC101
    connection=ConnectionRecord(
        backend=Backend.MSL,
        address='SDK::Thorlabs.MotionControl.KCube.Solenoid.dll',
    ),
)

MotionControl.build_device_list()

shutter = sh07.connect()
print('Connected to {}'.format(shutter))
shutter.set_operating_mode('Single')
shutter.set_cycle_params(19, 15, 1)

# Cube ------------------------------------------------------------------------
rotator = Thorlabs.KDC101("27001330")
print('Connected to {}'.format(rotator))

# Thorlabs ELLO Rotator ------------------------------------------------------------------------
ser = serial.Serial('COM4')
print('ELLO rotator connected to {}'.format(ser.name)) 

# Functions for optics ------------------------------------------------------------------------

def um(pos):
    return pos*34.304

def openShutter(pump, probe, filename):
    shutter.set_operating_state('Active')
    pulses_str = "{}\t{}\t{}".format(time.time(), pump, probe)
    with open(filename+'.pulses','a') as f: 
        f.write(pulses_str+"\n")
    time.sleep(0.1)
    shutter.set_operating_state('Inactive')

pump_calib = np.loadtxt('pumpPowerCalibration.dat')
probe_calib = np.loadtxt('probePowerCalibration.dat')

def MalusLaw(x, A, T, x0, off):
    return A*np.cos(2*np.pi*(x-x0)/T+np.pi/2)**2+off

def getIntensityPos(y, A, T, x0, off):
    return (T*np.arcsin(np.sqrt((y-off)/A)))/(2*np.pi)+x0

def readRot():
    ser.flushInput()
    ser.flushOutput()
    ser.write(b'0gp')
    r = ser.readline()
    return int((r.rstrip()).decode("utf-8")[3:], 16)

def writeRot(val):
    ser.flushInput()
    ser.flushOutput()
    w = '0ma'+format(val, 'X').zfill(8)
    ser.write(w.encode("utf-8"))

pump_popt, _ = curve_fit(MalusLaw, pump_calib[:,0], pump_calib[:,1], p0 = [np.max(pump_calib[:,1]),  2*(pump_calib[-1,0]-pump_calib[0,0]), (pump_calib[-1,0]+pump_calib[0,0])/2,  0])
probe_popt, _ = curve_fit(MalusLaw, probe_calib[:,0], probe_calib[:,1], p0 = [np.max(probe_calib[:,1]),  2*(probe_calib[-1,0]-probe_calib[0,0]), (probe_calib[-1,0]+probe_calib[0,0])/2,  0])

def pumpPower(power):
    power = power/1000 # z mW -> W 
    pos = getIntensityPos(power,*pump_popt)
    rotator.move_to(pos)
    rotator.wait_for_move()

def probePower(power):
    power = power/1000 # z mW -> W 
    pos = getIntensityPos(power,*probe_popt)
    writeRot(int(pos))
    time.sleep(0.5) 
      
def read_T():
    lakeshore.query("CRDG?")
    return(float(lakeshore.query("CRDG?")))


# main ------------------------------------------------------------------------
def measureDL(filename, pump, probe):

    spike_window = 30
    delta_t = 0.1
    stop = 50
    #filename = r"D:/P074n35/test1"
    
    pumpPower(pump)
    probePower(probe)
    time.sleep(2)
    
    sched = BackgroundScheduler()
    t0 = datetime.now()
    t_shift = timedelta(seconds=1)
    
    try:
        t_i = t0+timedelta(seconds=1)
        for _ in range(3):
            sched.add_job(openShutter, 'date', run_date=(t_i + t_shift), args=[pump, probe, filename])
            t_i = t_i + timedelta(seconds=10)
        
        sched.start()
    
        i=0
        Measure = True
        Spike = False
        afterSpike = 10 # in s; mer jeste {afterSpike} sekund po spiku
        
        
        instKSM.query(":READ?")
        time.sleep(0.2)
        measurement = np.zeros([1, 5])
        while Measure:
            
            r0 = time.time()
            r = instKSM.query(":READ?")
            r1 = float(r[0:13]) ### voltage
            r2 = float(r[14:27]) ### current
            r3 = float(instKMM.query("DATA?"))
            r4 = read_T()
            
            resstr = "{}\t{:.8e}\t{:.8e}\t{:.8e}\t{:.8e}".format(r0,r1,r2,r3,r4)
            measurement = np.append(measurement, [[r0,r1,r2,r3,r4]], axis=0)
            with open(filename+'.dat','a') as f: 
                f.write(resstr+"\n")
            
            # Detekce Spiku:
            if len(measurement) > spike_window:
                R4 = measurement[:,3]/ measurement[:,2]
                R4 = R4[-28:]
                R_max = np.max(R4)
                R_min = np.min(R4)
                R_avg = np.average(R4)
                signal = 100*(R_max-R_min)/(R_avg)
                #print("Spike: {} %".format(signal))
                if np.absolute(signal) > stop and not Spike:
                    sched.remove_all_jobs()
                    print('Spike was detected, next pulses halted')
                    logging.info('Spike was detected, next pulses halted')
                    Spike = True
            if Spike or len(sched.get_jobs())==0:
                afterSpike = afterSpike-delta_t
                print("spike:", afterSpike)
            if afterSpike<0:
                Measure = False
                print('Finished to measure last pulse')
                logging.info('Finished to measure last pulse')
                    
            # jednak zapisovani do file, ale taky do pole -> vzit okno z konce pole mensi nez vzdalenost mezi pulzy
            # measurement[0:zacatek] -> prumer => offset odporu
            # measurement[okno na konci] -> (max-min)/offset => velikost signalu v procentech
            # pokud presahne nejakou mez -> zastav
            time.sleep(delta_t)
        # precist pulses.pul a zjistit (pred)posledni a ten vzit jako zapisovaci
        sched.shutdown()
       
    except (KeyboardInterrupt, SystemExit):
        # Not strictly necessary if daemonic mode is enabled but should be done if possible
        sched.shutdown()
        
def run(start, end, step, pump, probe, folder):
    xps.move_stage(dline+'.Pos', start)
    time.sleep(5)
    for dl in np.arange(start, end, step):
        filename = folder+str(dl).replace(".", "_")
        xps.move_stage('Group1.Pos', dl)
        time.sleep(0.5)
        measureDL(filename, pump, probe)
    

SourceMeter:  KEITHLEY INSTRUMENTS INC.,MODEL 2400,4104420,C33   Mar 31 2015 09:32:39/A02  /U/K

MultiMeter:  KEITHLEY INSTRUMENTS INC.,MODEL 2400,4104420,C33   Mar 31 2015 09:32:39/A02  /U/K

LakeShore:  LSCI,MODEL335,335A0U4/#######,1.2

Connected to KCubeSolenoid<Thorlabs|KSC101|68000117>
Connected to <pylablib.aux_libs.devices.Thorlabs.KDC101 object at 0x0000020A32B7EE88>
ELLO rotator connected to COM4


In [2]:
run(200, 400, 0.5, 37, 15, r"D:/P074n35B2/data/experiment/")

C:\Users\TeraLab\anaconda3\lib\site-packages\ipykernel_launcher.py:177: RuntimeWarning: invalid value encountered in true_divide


spike: 9.9
spike: 9.8
spike: 9.700000000000001
spike: 9.600000000000001
spike: 9.500000000000002
spike: 9.400000000000002
spike: 9.300000000000002
spike: 9.200000000000003
spike: 9.100000000000003
spike: 9.000000000000004
spike: 8.900000000000004
spike: 8.800000000000004
spike: 8.700000000000005
spike: 8.600000000000005
spike: 8.500000000000005
spike: 8.400000000000006
spike: 8.300000000000006
spike: 8.200000000000006
spike: 8.100000000000007
spike: 8.000000000000007
spike: 7.9000000000000075
spike: 7.800000000000008
spike: 7.700000000000008
spike: 7.6000000000000085
spike: 7.500000000000009
spike: 7.400000000000009
spike: 7.30000000000001
spike: 7.20000000000001
spike: 7.10000000000001
spike: 7.000000000000011
spike: 6.900000000000011
spike: 6.800000000000011
spike: 6.700000000000012
spike: 6.600000000000012
spike: 6.500000000000012
spike: 6.400000000000013
spike: 6.300000000000013
spike: 6.2000000000000135
spike: 6.100000000000014
spike: 6.000000000000014
spike: 5.900000000000015
spi

spike: 7.700000000000008
spike: 7.6000000000000085
spike: 7.500000000000009
spike: 7.400000000000009
spike: 7.30000000000001
spike: 7.20000000000001
spike: 7.10000000000001
spike: 7.000000000000011
spike: 6.900000000000011
spike: 6.800000000000011
spike: 6.700000000000012
spike: 6.600000000000012
spike: 6.500000000000012
spike: 6.400000000000013
spike: 6.300000000000013
spike: 6.2000000000000135
spike: 6.100000000000014
spike: 6.000000000000014
spike: 5.900000000000015
spike: 5.800000000000015
spike: 5.700000000000015
spike: 5.600000000000016
spike: 5.500000000000016
spike: 5.400000000000016
spike: 5.300000000000017
spike: 5.200000000000017
spike: 5.100000000000017
spike: 5.000000000000018
spike: 4.900000000000018
spike: 4.8000000000000185
spike: 4.700000000000019
spike: 4.600000000000019
spike: 4.5000000000000195
spike: 4.40000000000002
spike: 4.30000000000002
spike: 4.200000000000021
spike: 4.100000000000021
spike: 4.000000000000021
spike: 3.9000000000000212
spike: 3.800000000000021


spike: 5.600000000000016
spike: 5.500000000000016
spike: 5.400000000000016
spike: 5.300000000000017
spike: 5.200000000000017
spike: 5.100000000000017
spike: 5.000000000000018
spike: 4.900000000000018
spike: 4.8000000000000185
spike: 4.700000000000019
spike: 4.600000000000019
spike: 4.5000000000000195
spike: 4.40000000000002
spike: 4.30000000000002
spike: 4.200000000000021
spike: 4.100000000000021
spike: 4.000000000000021
spike: 3.9000000000000212
spike: 3.800000000000021
spike: 3.700000000000021
spike: 3.600000000000021
spike: 3.500000000000021
spike: 3.400000000000021
spike: 3.3000000000000207
spike: 3.2000000000000206
spike: 3.1000000000000205
spike: 3.0000000000000204
spike: 2.9000000000000203
spike: 2.8000000000000203
spike: 2.70000000000002
spike: 2.60000000000002
spike: 2.50000000000002
spike: 2.40000000000002
spike: 2.30000000000002
spike: 2.2000000000000197
spike: 2.1000000000000196
spike: 2.0000000000000195
spike: 1.9000000000000195
spike: 1.8000000000000194
spike: 1.700000000

spike: 3.400000000000021
spike: 3.3000000000000207
spike: 3.2000000000000206
spike: 3.1000000000000205
spike: 3.0000000000000204
spike: 2.9000000000000203
spike: 2.8000000000000203
spike: 2.70000000000002
spike: 2.60000000000002
spike: 2.50000000000002
spike: 2.40000000000002
spike: 2.30000000000002
spike: 2.2000000000000197
spike: 2.1000000000000196
spike: 2.0000000000000195
spike: 1.9000000000000195
spike: 1.8000000000000194
spike: 1.7000000000000193
spike: 1.6000000000000192
spike: 1.500000000000019
spike: 1.400000000000019
spike: 1.300000000000019
spike: 1.2000000000000188
spike: 1.1000000000000187
spike: 1.0000000000000187
spike: 0.9000000000000187
spike: 0.8000000000000187
spike: 0.7000000000000187
spike: 0.6000000000000187
spike: 0.5000000000000188
spike: 0.4000000000000188
spike: 0.3000000000000188
spike: 0.2000000000000188
spike: 0.1000000000000188
spike: 1.8790524691780774e-14
spike: -0.09999999999998122
Finished to measure last pulse
spike: 9.9
spike: 9.8
spike: 9.7000000000

spike: 1.2000000000000188
spike: 1.1000000000000187
spike: 1.0000000000000187
spike: 0.9000000000000187
spike: 0.8000000000000187
spike: 0.7000000000000187
spike: 0.6000000000000187
spike: 0.5000000000000188
spike: 0.4000000000000188
spike: 0.3000000000000188
spike: 0.2000000000000188
spike: 0.1000000000000188
spike: 1.8790524691780774e-14
spike: -0.09999999999998122
Finished to measure last pulse
spike: 9.9
spike: 9.8
spike: 9.700000000000001
spike: 9.600000000000001
spike: 9.500000000000002
spike: 9.400000000000002
spike: 9.300000000000002
spike: 9.200000000000003
spike: 9.100000000000003
spike: 9.000000000000004
spike: 8.900000000000004
spike: 8.800000000000004
spike: 8.700000000000005
spike: 8.600000000000005
spike: 8.500000000000005
spike: 8.400000000000006
spike: 8.300000000000006
spike: 8.200000000000006
spike: 8.100000000000007
spike: 8.000000000000007
spike: 7.9000000000000075
spike: 7.800000000000008
spike: 7.700000000000008
spike: 7.6000000000000085
spike: 7.500000000000009


spike: 9.300000000000002
spike: 9.200000000000003
spike: 9.100000000000003
spike: 9.000000000000004
spike: 8.900000000000004
spike: 8.800000000000004
spike: 8.700000000000005
spike: 8.600000000000005
spike: 8.500000000000005
spike: 8.400000000000006
spike: 8.300000000000006
spike: 8.200000000000006
spike: 8.100000000000007
spike: 8.000000000000007
spike: 7.9000000000000075
spike: 7.800000000000008
spike: 7.700000000000008
spike: 7.6000000000000085
spike: 7.500000000000009
spike: 7.400000000000009
spike: 7.30000000000001
spike: 7.20000000000001
spike: 7.10000000000001
spike: 7.000000000000011
spike: 6.900000000000011
spike: 6.800000000000011
spike: 6.700000000000012
spike: 6.600000000000012
spike: 6.500000000000012
spike: 6.400000000000013
spike: 6.300000000000013
spike: 6.2000000000000135
spike: 6.100000000000014
spike: 6.000000000000014
spike: 5.900000000000015
spike: 5.800000000000015
spike: 5.700000000000015
spike: 5.600000000000016
spike: 5.500000000000016
spike: 5.400000000000016


spike: 7.20000000000001
spike: 7.10000000000001
spike: 7.000000000000011
spike: 6.900000000000011
spike: 6.800000000000011
spike: 6.700000000000012
spike: 6.600000000000012
spike: 6.500000000000012
spike: 6.400000000000013
spike: 6.300000000000013
spike: 6.2000000000000135
spike: 6.100000000000014
spike: 6.000000000000014
spike: 5.900000000000015
spike: 5.800000000000015
spike: 5.700000000000015
spike: 5.600000000000016
spike: 5.500000000000016
spike: 5.400000000000016
spike: 5.300000000000017
spike: 5.200000000000017
spike: 5.100000000000017
spike: 5.000000000000018
spike: 4.900000000000018
spike: 4.8000000000000185
spike: 4.700000000000019
spike: 4.600000000000019
spike: 4.5000000000000195
spike: 4.40000000000002
spike: 4.30000000000002
spike: 4.200000000000021
spike: 4.100000000000021
spike: 4.000000000000021
spike: 3.9000000000000212
spike: 3.800000000000021
spike: 3.700000000000021
spike: 3.600000000000021
spike: 3.500000000000021
spike: 3.400000000000021
spike: 3.3000000000000207

In [2]:
pump = 37
probe = 15
folder = r"D:/P074n35B2/data/experiment/"

xps.move_stage(dline+'.Pos', -400)
time.sleep(5)
for dl in np.arange(-400, 0, 10):
    filename = folder+str(dl).replace(".", "_")
    xps.move_stage('Group1.Pos', dl)
    time.sleep(0.5)
    measureDL(filename, pump, probe)
    
xps.move_stage(dline+'.Pos', 200)
time.sleep(5)
for dl in np.arange(0, 250, 2.5):
    filename = folder+str(dl).replace(".", "_")
    xps.move_stage('Group1.Pos', dl)
    time.sleep(0.5)
    measureDL(filename, pump, probe)
    
xps.move_stage(dline+'.Pos', 250)
time.sleep(5)
for dl in np.arange(250, 400, 0.5):
    filename = folder+str(dl).replace(".", "_")
    xps.move_stage('Group1.Pos', dl)
    time.sleep(0.5)
    measureDL(filename, pump, probe)

C:\Users\TeraLab\anaconda3\lib\site-packages\ipykernel_launcher.py:177: RuntimeWarning: invalid value encountered in true_divide


spike: 9.9
spike: 9.8
spike: 9.700000000000001
spike: 9.600000000000001
spike: 9.500000000000002
spike: 9.400000000000002
spike: 9.300000000000002
spike: 9.200000000000003
spike: 9.100000000000003
spike: 9.000000000000004
spike: 8.900000000000004
spike: 8.800000000000004
spike: 8.700000000000005
spike: 8.600000000000005
spike: 8.500000000000005
spike: 8.400000000000006
spike: 8.300000000000006
spike: 8.200000000000006
spike: 8.100000000000007
spike: 8.000000000000007
spike: 7.9000000000000075
spike: 7.800000000000008
spike: 7.700000000000008
spike: 7.6000000000000085
spike: 7.500000000000009
spike: 7.400000000000009
spike: 7.30000000000001
spike: 7.20000000000001
spike: 7.10000000000001
spike: 7.000000000000011
spike: 6.900000000000011
spike: 6.800000000000011
spike: 6.700000000000012
spike: 6.600000000000012
spike: 6.500000000000012
spike: 6.400000000000013
spike: 6.300000000000013
spike: 6.2000000000000135
spike: 6.100000000000014
spike: 6.000000000000014
spike: 5.900000000000015
spi

spike: 7.700000000000008
spike: 7.6000000000000085
spike: 7.500000000000009
spike: 7.400000000000009
spike: 7.30000000000001
spike: 7.20000000000001
spike: 7.10000000000001
spike: 7.000000000000011
spike: 6.900000000000011
spike: 6.800000000000011
spike: 6.700000000000012
spike: 6.600000000000012
spike: 6.500000000000012
spike: 6.400000000000013
spike: 6.300000000000013
spike: 6.2000000000000135
spike: 6.100000000000014
spike: 6.000000000000014
spike: 5.900000000000015
spike: 5.800000000000015
spike: 5.700000000000015
spike: 5.600000000000016
spike: 5.500000000000016
spike: 5.400000000000016
spike: 5.300000000000017
spike: 5.200000000000017
spike: 5.100000000000017
spike: 5.000000000000018
spike: 4.900000000000018
spike: 4.8000000000000185
spike: 4.700000000000019
spike: 4.600000000000019
spike: 4.5000000000000195
spike: 4.40000000000002
spike: 4.30000000000002
spike: 4.200000000000021
spike: 4.100000000000021
spike: 4.000000000000021
spike: 3.9000000000000212
spike: 3.800000000000021


spike: 5.600000000000016
spike: 5.500000000000016
spike: 5.400000000000016
spike: 5.300000000000017
spike: 5.200000000000017
spike: 5.100000000000017
spike: 5.000000000000018
spike: 4.900000000000018
spike: 4.8000000000000185
spike: 4.700000000000019
spike: 4.600000000000019
spike: 4.5000000000000195
spike: 4.40000000000002
spike: 4.30000000000002
spike: 4.200000000000021
spike: 4.100000000000021
spike: 4.000000000000021
spike: 3.9000000000000212
spike: 3.800000000000021
spike: 3.700000000000021
spike: 3.600000000000021
spike: 3.500000000000021
spike: 3.400000000000021
spike: 3.3000000000000207
spike: 3.2000000000000206
spike: 3.1000000000000205
spike: 3.0000000000000204
spike: 2.9000000000000203
spike: 2.8000000000000203
spike: 2.70000000000002
spike: 2.60000000000002
spike: 2.50000000000002
spike: 2.40000000000002
spike: 2.30000000000002
spike: 2.2000000000000197
spike: 2.1000000000000196
spike: 2.0000000000000195
spike: 1.9000000000000195
spike: 1.8000000000000194
spike: 1.700000000

spike: 3.500000000000021
spike: 3.400000000000021
spike: 3.3000000000000207
spike: 3.2000000000000206
spike: 3.1000000000000205
spike: 3.0000000000000204
spike: 2.9000000000000203
spike: 2.8000000000000203
spike: 2.70000000000002
spike: 2.60000000000002
spike: 2.50000000000002
spike: 2.40000000000002
spike: 2.30000000000002
spike: 2.2000000000000197
spike: 2.1000000000000196
spike: 2.0000000000000195
spike: 1.9000000000000195
spike: 1.8000000000000194
spike: 1.7000000000000193
spike: 1.6000000000000192
spike: 1.500000000000019
spike: 1.400000000000019
spike: 1.300000000000019
spike: 1.2000000000000188
spike: 1.1000000000000187
spike: 1.0000000000000187
spike: 0.9000000000000187
spike: 0.8000000000000187
spike: 0.7000000000000187
spike: 0.6000000000000187
spike: 0.5000000000000188
spike: 0.4000000000000188
spike: 0.3000000000000188
spike: 0.2000000000000188
spike: 0.1000000000000188
spike: 1.8790524691780774e-14
spike: -0.09999999999998122
Finished to measure last pulse
spike: 9.9
spike

...

spike: 1.300000000000019
spike: 1.2000000000000188
spike: 1.1000000000000187
spike: 1.0000000000000187
spike: 0.9000000000000187
spike: 0.8000000000000187
spike: 0.7000000000000187
spike: 0.6000000000000187
spike: 0.5000000000000188
spike: 0.4000000000000188
spike: 0.3000000000000188
spike: 0.2000000000000188
spike: 0.1000000000000188
spike: 1.8790524691780774e-14
spike: -0.09999999999998122
Finished to measure last pulse
spike: 9.9
spike: 9.8
spike: 9.700000000000001
spike: 9.600000000000001
spike: 9.500000000000002
spike: 9.400000000000002
spike: 9.300000000000002
spike: 9.200000000000003
spike: 9.100000000000003
spike: 9.000000000000004
spike: 8.900000000000004
spike: 8.800000000000004
spike: 8.700000000000005
spike: 8.600000000000005
spike: 8.500000000000005
spike: 8.400000000000006
spike: 8.300000000000006
spike: 8.200000000000006
spike: 8.100000000000007
spike: 8.000000000000007
spike: 7.9000000000000075
spike: 7.800000000000008
spike: 7.700000000000008
spike: 7.6000000000000085


spike: 9.400000000000002
spike: 9.300000000000002
spike: 9.200000000000003
spike: 9.100000000000003
spike: 9.000000000000004
spike: 8.900000000000004
spike: 8.800000000000004
spike: 8.700000000000005
spike: 8.600000000000005
spike: 8.500000000000005
spike: 8.400000000000006
spike: 8.300000000000006
spike: 8.200000000000006
spike: 8.100000000000007
spike: 8.000000000000007
spike: 7.9000000000000075
spike: 7.800000000000008
spike: 7.700000000000008
spike: 7.6000000000000085
spike: 7.500000000000009
spike: 7.400000000000009
spike: 7.30000000000001
spike: 7.20000000000001
spike: 7.10000000000001
spike: 7.000000000000011
spike: 6.900000000000011
spike: 6.800000000000011
spike: 6.700000000000012
spike: 6.600000000000012
spike: 6.500000000000012
spike: 6.400000000000013
spike: 6.300000000000013
spike: 6.2000000000000135
spike: 6.100000000000014
spike: 6.000000000000014
spike: 5.900000000000015
spike: 5.800000000000015
spike: 5.700000000000015
spike: 5.600000000000016
spike: 5.500000000000016


spike: 7.20000000000001
spike: 7.10000000000001
spike: 7.000000000000011
spike: 6.900000000000011
spike: 6.800000000000011
spike: 6.700000000000012
spike: 6.600000000000012
spike: 6.500000000000012
spike: 6.400000000000013
spike: 6.300000000000013
spike: 6.2000000000000135
spike: 6.100000000000014
spike: 6.000000000000014
spike: 5.900000000000015
spike: 5.800000000000015
spike: 5.700000000000015
spike: 5.600000000000016
spike: 5.500000000000016
spike: 5.400000000000016
spike: 5.300000000000017
spike: 5.200000000000017
spike: 5.100000000000017
spike: 5.000000000000018
spike: 4.900000000000018
spike: 4.8000000000000185
spike: 4.700000000000019
spike: 4.600000000000019
spike: 4.5000000000000195
spike: 4.40000000000002
spike: 4.30000000000002
spike: 4.200000000000021
spike: 4.100000000000021
spike: 4.000000000000021
spike: 3.9000000000000212
spike: 3.800000000000021
spike: 3.700000000000021
spike: 3.600000000000021
spike: 3.500000000000021
spike: 3.400000000000021
spike: 3.3000000000000207

spike: 5.100000000000017
spike: 5.000000000000018
spike: 4.900000000000018
spike: 4.8000000000000185
spike: 4.700000000000019
spike: 4.600000000000019
spike: 4.5000000000000195
spike: 4.40000000000002
spike: 4.30000000000002
spike: 4.200000000000021
spike: 4.100000000000021
spike: 4.000000000000021
spike: 3.9000000000000212
spike: 3.800000000000021
spike: 3.700000000000021
spike: 3.600000000000021
spike: 3.500000000000021
spike: 3.400000000000021
spike: 3.3000000000000207
spike: 3.2000000000000206
spike: 3.1000000000000205
spike: 3.0000000000000204
spike: 2.9000000000000203
spike: 2.8000000000000203
spike: 2.70000000000002
spike: 2.60000000000002
spike: 2.50000000000002
spike: 2.40000000000002
spike: 2.30000000000002
spike: 2.2000000000000197
spike: 2.1000000000000196
spike: 2.0000000000000195
spike: 1.9000000000000195
spike: 1.8000000000000194
spike: 1.7000000000000193
spike: 1.6000000000000192
spike: 1.500000000000019
spike: 1.400000000000019
spike: 1.300000000000019
spike: 1.2000000

spike: 3.0000000000000204
spike: 2.9000000000000203
spike: 2.8000000000000203
spike: 2.70000000000002
spike: 2.60000000000002
spike: 2.50000000000002
spike: 2.40000000000002
spike: 2.30000000000002
spike: 2.2000000000000197
spike: 2.1000000000000196
spike: 2.0000000000000195
spike: 1.9000000000000195
spike: 1.8000000000000194
spike: 1.7000000000000193
spike: 1.6000000000000192
spike: 1.500000000000019
spike: 1.400000000000019
spike: 1.300000000000019
spike: 1.2000000000000188
spike: 1.1000000000000187
spike: 1.0000000000000187
spike: 0.9000000000000187
spike: 0.8000000000000187
spike: 0.7000000000000187
spike: 0.6000000000000187
spike: 0.5000000000000188
spike: 0.4000000000000188
spike: 0.3000000000000188
spike: 0.2000000000000188
spike: 0.1000000000000188
spike: 1.8790524691780774e-14
spike: -0.09999999999998122
Finished to measure last pulse


In [3]:
import numpy as np
np.arange(-400, 200, 10)

array([-400, -390, -380, -370, -360, -350, -340, -330, -320, -310, -300,
       -290, -280, -270, -260, -250, -240, -230, -220, -210, -200, -190,
       -180, -170, -160, -150, -140, -130, -120, -110, -100,  -90,  -80,
        -70,  -60,  -50,  -40,  -30,  -20,  -10,    0,   10,   20,   30,
         40,   50,   60,   70,   80,   90,  100,  110,  120,  130,  140,
        150,  160,  170,  180,  190])

In [5]:
# Newport Delay Line --------------------------------------------------------------
from newportxps import NewportXPS
xps = NewportXPS('192.168.254.254', username='Administrator', password='Administrator')
dline = 'Group1'
xps.initialize_group(group=dline, home=True)

In [6]:
xps.move_stage(dline+'.Pos', 200)

''